In [2]:
import Functions2
import cv2
import numpy as np
from math import sqrt
from skimage import color, exposure
from skimage.color import rgb2gray
from skimage.feature import blob_log
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from sklearn.decomposition import PCA


In [5]:
# Load and process the image
image = cv2.imread(r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Test\PAT_56_86_479.png')

import cv2
import numpy as np
from math import sqrt
from skimage import color, exposure
from skimage.color import rgb2gray
from skimage.feature import blob_log
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from sklearn.decomposition import PCA

# Function to perform PCA on a given feature vector
def apply_pca(feature_vector):
    pca = PCA(n_components=2)  # You can adjust the number of components as per your requirement
    transformed = pca.fit_transform(feature_vector)
    return transformed

# Function to measure pigment network
def measure_pigment_network(image):
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_channel, _, _ = cv2.split(lab_image)

    enhanced_l_channel = cv2.equalizeHist(l_channel)
    _, binary_mask = cv2.threshold(enhanced_l_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    total_pixels = np.prod(binary_mask.shape[:2])
    pigment_pixels = np.count_nonzero(binary_mask)
    coverage_percentage = (pigment_pixels / total_pixels) * 100

    return np.array([[coverage_percentage]])  # Reshape the scalar value into a 2D array


# Function to measure blue veil
def measure_blue_veil(image):
    height, width, _ = image.shape
    count = 0

    for y in range(height):
        for x in range(width):
            b, g, r = image[y, x]

            if b > 60 and (r - 46 < g) and (g < r + 15):
                count += 1

    return np.array([[count]])  # Reshape the scalar value into a 2D array


# Function to measure vascular
def measure_vascular(image):
    red_channel = image[:, :, 0]
    enhanced_red_channel = exposure.adjust_gamma(red_channel, gamma=1)
    enhanced_image = image.copy()
    enhanced_image[:, :, 0] = enhanced_red_channel
    hsv_img = color.rgb2hsv(enhanced_image)

    lower_red1 = np.array([0, 40/100, 00/100])
    upper_red1 = np.array([25/360, 1, 1])
    mask1 = np.logical_and(np.all(hsv_img >= lower_red1, axis=-1), np.all(hsv_img <= upper_red1, axis=-1))

    lower_red2 = np.array([330/360, 40/100, 00/100])  # Lower limit for red hue, saturation, and value
    upper_red2 = np.array([1, 1, 1])  # Upper limit for red hue, saturation, and value
    mask2 = np.logical_and(np.all(hsv_img >= lower_red2, axis=-1), np.all(hsv_img <= upper_red2, axis=-1))

    mask = np.logical_or(mask1, mask2)

    return np.array([[np.sum(mask)]])  # Reshape the scalar value into a 2D array


# Function to measure globules
def measure_globules(image):
    image_gray = rgb2gray(image)
    inverted_image = 1 - image_gray

    blobs_doh = blob_log(inverted_image, min_sigma=1, max_sigma=4, num_sigma=50, threshold=.05)
    blobs_doh[:, 2] = blobs_doh[:, 2] * sqrt(2)
    blob_amount = len(blobs_doh)

    return np.array([[blob_amount]])  # Reshape the scalar value into a 2D array


# Function to measure streaks
def measure_streaks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    lesion_area = cv2.contourArea(contours[0])
    border_perimeter = cv2.arcLength(contours[0], True)
    if lesion_area == 0:
        irregularity = 0 
    else:
        irregularity = (border_perimeter ** 2) / (4 * np.pi * lesion_area)

    return np.array([[irregularity]])  # Reshape the scalar value into a 2D array


# Function to measure irregular pigmentation
def measure_irregular_pigmentation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    threshold = threshold_otsu(gray)
    binary = gray > threshold
    labeled_image = label(binary)

    min_rows, min_cols, max_rows, max_cols = [], [], [], []

    for region in regionprops(labeled_image):
        area = region.area
        perimeter = region.perimeter

        if perimeter == 0:
            continue

        circularity = 4 * np.pi * (area / (perimeter ** 2))

        if circularity < 0.6:
            min_row, min_col, max_row, max_col = region.bbox
            min_rows.append(min_row)
            min_cols.append(min_col)
            max_rows.append(max_row)
            max_cols.append(max_col)

    _, binary_mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    total_pixels = np.prod(binary_mask.shape[:2])
    irregular_pixels = np.count_nonzero(binary_mask)
    coverage_percentage = (irregular_pixels / total_pixels) * 100

    return np.array([[coverage_percentage]])  # Reshape the scalar value into a 2D array


# Function to measure regression
def measure_regression(image):
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_color = np.array([0, 0, 150])
    upper_color = np.array([180, 30, 255])
    mask = cv2.inRange(hsv_img, lower_color, upper_color)
    num_pixels = cv2.countNonZero(mask)

    return np.array([[num_pixels]])  # Reshape the scalar value into a 2D array


In [8]:
# Load the image
image = cv2.imread(r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Test\PAT_998_17_641.png')

# Compute feature vectors
pigment_network_feature = measure_pigment_network(image)
blue_veil_feature = measure_blue_veil(image)
vascular_feature = measure_vascular(image)
globules_feature = measure_globules(image)
streaks_feature = measure_streaks(image)
irregular_pigmentation_feature = measure_irregular_pigmentation(image)
regression_feature = measure_regression(image)

# Apply PCA to each feature vector
pigment_network_pca = apply_pca(pigment_network_feature)
blue_veil_pca = apply_pca(blue_veil_feature)
vascular_pca = apply_pca(vascular_feature)
globules_pca = apply_pca(globules_feature)
streaks_pca = apply_pca(streaks_feature)
irregular_pigmentation_pca = apply_pca(irregular_pigmentation_feature)
regression_pca = apply_pca(regression_feature)

# Print the transformed feature vectors after PCA
print("Pigment Network PCA:")
print(pigment_network_pca)

print("Blue Veil PCA:")
print(blue_veil_pca)

print("Vascular PCA:")
print(vascular_pca)

print("Globules PCA:")
print(globules_pca)

print("Streaks PCA:")
print(streaks_pca)

print("Irregular Pigmentation PCA:")
print(irregular_pigmentation_pca)

print("Regression PCA:")
print(regression_pca)


ValueError: n_components=2 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

In [9]:
import cv2
import numpy as np
from math import sqrt
from skimage import color, exposure
from skimage.color import rgb2gray
from skimage.feature import blob_log
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops

# Function to measure pigment network
def measure_pigment_network(image):
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_channel, _, _ = cv2.split(lab_image)

    enhanced_l_channel = cv2.equalizeHist(l_channel)
    _, binary_mask = cv2.threshold(enhanced_l_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    total_pixels = np.prod(binary_mask.shape[:2])
    pigment_pixels = np.count_nonzero(binary_mask)
    coverage_percentage = (pigment_pixels / total_pixels) * 100

    return coverage_percentage

# Function to measure blue veil
def measure_blue_veil(image):
    height, width, _ = image.shape
    count = 0

    for y in range(height):
        for x in range(width):
            b, g, r = image[y, x]

            if b > 60 and (r - 46 < g) and (g < r + 15):
                count += 1

    return count

# Function to measure vascular
def measure_vascular(image):
    red_channel = image[:, :, 0]
    enhanced_red_channel = exposure.adjust_gamma(red_channel, gamma=1)
    enhanced_image = image.copy()
    enhanced_image[:, :, 0] = enhanced_red_channel
    hsv_img = color.rgb2hsv(enhanced_image)

    lower_red1 = np.array([0, 40/100, 00/100])
    upper_red1 = np.array([25/360, 1, 1])
    mask1 = np.logical_and(np.all(hsv_img >= lower_red1, axis=-1), np.all(hsv_img <= upper_red1, axis=-1))

    lower_red2 = np.array([330/360, 40/100, 00/100])  # Lower limit for red hue, saturation, and value
    upper_red2 = np.array([1, 1, 1])  # Upper limit for red hue, saturation, and value
    mask2 = np.logical_and(np.all(hsv_img >= lower_red2, axis=-1), np.all(hsv_img <= upper_red2, axis=-1))

    mask = np.logical_or(mask1, mask2)

    return np.sum(mask)

# Function to measure globules
def measure_globules(image):
    image_gray = rgb2gray(image)
    inverted_image = 1 - image_gray

    blobs_doh = blob_log(inverted_image, min_sigma=1, max_sigma=4, num_sigma=50, threshold=.05)
    blobs_doh[:, 2] = blobs_doh[:, 2] * sqrt(2)
    blob_amount = len(blobs_doh)

    return blob_amount

# Function to measure streaks
def measure_streaks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    lesion_area = cv2.contourArea(contours[0])
    border_perimeter = cv2.arcLength(contours[0], True)
    if lesion_area == 0:
        irregularity = 0
    else:
        irregularity = (border_perimeter ** 2) / (4 * np.pi * lesion_area)

    return irregularity

# Function to measure irregular pigmentation
def measure_irregular_pigmentation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    threshold = threshold_otsu(gray)
    binary = gray > threshold
    labeled_image = label(binary)

    min_rows, min_cols, max_rows, max_cols = [], [], [], []

    for region in regionprops(labeled_image):
        area = region.area
        perimeter = region.perimeter

        if perimeter == 0:
            continue

        circularity = 4 * np.pi * (area / (perimeter ** 2))

        if circularity < 0.6:
            min_row, min_col, max_row, max_col = region.bbox
            min_rows.append(min_row)
            min_cols.append(min_col)
            max_rows.append(max_row)
            max_cols.append(max_col)

    _, binary_mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    total_pixels = np.prod(binary_mask.shape[:2])
    irregular_pixels = np.count_nonzero(binary_mask)
    coverage_percentage = (irregular_pixels / total_pixels) * 100

    return coverage_percentage

# Function to measure regression
def measure_regression(image):
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_color = np.array([0, 0, 150])
    upper_color = np.array([180, 30, 255])
    mask = cv2.inRange(hsv_img, lower_color, upper_color)
    num_pixels = cv2.countNonZero(mask)

    return num_pixels

# Load the image
image = cv2.imread(r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Test\PAT_270_417_257.png')

# Compute feature vectors
pigment_network_feature = measure_pigment_network(image)
blue_veil_feature = measure_blue_veil(image)
vascular_feature = measure_vascular(image)
globules_feature = measure_globules(image)
streaks_feature = measure_streaks(image)
irregular_pigmentation_feature = measure_irregular_pigmentation(image)
regression_feature = measure_regression(image)

# Print the feature vectors
print("Pigment Network Feature:")
print(pigment_network_feature)

print("Blue Veil Feature:")
print(blue_veil_feature)

print("Vascular Feature:")
print(vascular_feature)

print("Globules Feature:")
print(globules_feature)

print("Streaks Feature:")
print(streaks_feature)

print("Irregular Pigmentation Feature:")
print(irregular_pigmentation_feature)

print("Regression Feature:")
print(regression_feature)


Pigment Network Feature:
18.05419921875
Blue Veil Feature:
3899
Vascular Feature:
0
Globules Feature:
80
Streaks Feature:
1.2732395447351628
Irregular Pigmentation Feature:
25.82244873046875
Regression Feature:
0


In [ ]:
import csv
from sklearn.feature_selection import mutual_info_classif
import numpy as np

# Function to extract features from an image
def extract_features(image):
    # Call your feature measurement functions here and return a feature vector
    feature_vector = [
        measure_pigment_network(image),
        measure_blue_veil(image),
        measure_vascular(image),
        measure_globules(image),
        measure_streaks(image),
        measure_irregular_pigmentation(image),
        measure_regression(image)
    ]
    return feature_vector

# Read image names and labels from the CSV file
csv_file = r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\full_data.csv'  # Replace with your CSV file name
image_names = []
labels = []
with open(csv_file, "r") as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        image_names.append(row[0])  # Assuming image names are in the first column
        labels.append(row[1])  # Assuming labels are in the second column

# Example usage
images = []
for image_name in image_names:
    # Load your images based on the image names and append them to the images list
    image = cv2.imread(image_name)
    images.append(image)

feature_matrix = []
for image in images:
    features = extract_features(image)
    feature_matrix.append(features)

X = np.array(feature_matrix)
y = np.array(labels)

mi_scores = mutual_info_classif(X, y)
for i, mi in enumerate(mi_scores):
    print(f"Feature {i+1}: {mi}")


In [12]:
import cv2
import numpy as np
from math import sqrt
from skimage import color, exposure
from skimage.color import rgb2gray
from skimage.feature import blob_log
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Function to perform LDA on the features
def perform_lda(features):
    # Convert the list of features into a numpy array
    X = np.array(features)

    # Perform LDA
    lda = LinearDiscriminantAnalysis(n_components=2)
    X_lda = lda.fit_transform(X)

    # Return the transformed features
    return X_lda

# Example usage
image1 = cv2.imread(r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Test\PAT_333_1469_499.png')
image2 = cv2.imread(r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Test\PAT_38_54_234.png')

feature1 = [
    measure_pigment_network(image1),
    measure_blue_veil(image1),
    measure_vascular(image1),
    measure_globules(image1),
    measure_streaks(image1),
    measure_irregular_pigmentation(image1),
    measure_regression(image1)
]

feature2 = [
    measure_pigment_network(image2),
    measure_blue_veil(image2),
    measure_vascular(image2),
    measure_globules(image2),
    measure_streaks(image2),
    measure_irregular_pigmentation(image2),
    measure_regression(image2)
]

features = [feature1, feature2]

# Perform LDA on the features
transformed_features = perform_lda(features)

# Print the transformed features
print("Transformed features:")
print(transformed_features)


TypeError: fit() missing 1 required positional argument: 'y'